# The Battle of Neighborhoods

### Introduction / Business Problem definition

In the US, one dense area is known to be highly different from the rest of the country in terms of innovation. The San Francisco Bay Area is indeed the most innovative place in the world, gathering more than 50% of all VC funds in the world, in a single, small region. 
The most important and famous parts of the Bay area include cities like San Francisco, Berkeley, San Jose, Palo Alto ... but the region is not limited to these territories and also include smaller cities a few kilometers away from the center of the Bay Area. 

Most of the highly successful tech companies were launched in this area and the highest paying jobs are located in that particular area. The San Francisco Bay Area is also well known for its diversity. In the area, it is often said that one is not from San Francisco if he lived there for more than a couple of years. 

Based on these few observations, we noticed an important problem that was created in the Bay Area: because of the average salary and the quality of life in this area, housing prices are completely disproportionate and for a new comer in the area, it can be a crazy riddle to solve when it comes to finding a place to live. 

In this analysis, I will get to know more about the Bay Area, all its locations and its main venues. Based on that, we will try to identify areas that can be similar (or not) to the city center of San Francisco, so that a new comer can easily identify where would be the best place for him to live in based on his preferences and salary. Throughout this analysis, we will try to understand why is Silicon Valley so different from the rest of the world and the rest of the Bay Area / California.

### Data

#### Data Sources

In order to achieve this analysis, I will need three types of information : 
- Information regarding the cities in The Bay Area, their postal codes, their associated coordinates (latitude and longitude). This information should be easily found on governmental websites : https://catalog.data.gov/dataset/bay-area-zip-codes/resource/6cacd1a1-6bff-4c7c-9094-49188ea29f85
- Associated venue to each of the different cities and neighborhoods, using Foursquare and its API.
- The San Francisco JSON file which provides a list of all Bay Area Boroughs, Neighborhoods, and their coordinates (latitude and longitude), if we cannot find it using the governmental websites information

### Methodology 

Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.

Our methodology will be divided in different parts : 
1. Import packages and tools: In this section, we will import all tools and packages that we will require throughout the analysis. Those include but is not limited to: CSV, PD, NP, REQUESTS, KMEANS, JSON, GEOCODER, MATPLOTLIB, GEOCODER, FOLIUM, etc.
2. Data cleaning : Using the governmental website link that we found, let's import the CSV / HTML format in order to put it into a pandas dataframe. Once the required data is put into a pandas dataframe, we will be required to delete all the unecessary rows and columns, rename columns, add BLANK columns for latitude & longitude... The objective would be to have a table with columns for Neighborhood, ZIP code, Latitude and Longitude 
3. Use Geopy library: With Geopy library, we will be able to associate Coordinates (Latitude & Longitude) to the ZIP code. 
4. Use the FourSquare API: With Foursquare credentials, import all venues associated to the different locations. 
5. Once all the data is being collected and associated in one grouped table, run the Kmeans Machine Learning algorithm in order to determine what are the main clusters. Iterate based on the number of clusters required to get a good understanding of the data

#### Import & Data Cleaning

First let's import some key tools that will be useful throughout our analysis, these include but is not limited to numpy, pandas, jason ... 

In [1]:
import csv
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Install geocoder
!pip install geocoder
import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip install folium
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 98 kB 3.3 MB/s eta 0:00:011
     |████████████████████████████████| 93 kB 3.3 MB/s  eta 0:00:01
Libraries imported.


In [2]:
df = pd.read_csv ('https://data.sfgov.org/api/views/f9wk-m4qb/rows.csv?accessType=DOWNLOAD')   
df.head(10)

,PO_NAME,the_geom,ZIP,STATE,Area__,Length__
0,NAPA,MULTIPOLYGON (((-122.10329200180091 38.5132829...,94558,CA,1.231326e+10,995176.225313
1,FAIRFIELD,MULTIPOLYGON (((-121.947475002335 38.301511000...,94533,CA,9.917861e+08,200772.556587
2,DIXON,MULTIPOLYGON (((-121.65335500334429 38.3133870...,95620,CA,7.236950e+09,441860.201400
3,SONOMA,MULTIPOLYGON (((-122.406843003057 38.155681999...,95476,CA,3.001414e+09,311318.546326
4,NAPA,MULTIPOLYGON (((-122.29368500225117 38.1552379...,94559,CA,1.194302e+09,359104.646602
5,PETALUMA,MULTIPOLYGON (((-122.45766900253919 38.1168949...,94954,CA,2.006544e+09,267474.490552
6,RIO VISTA,MULTIPOLYGON (((-121.8624620022998 38.06602999...,94571,CA,4.454446e+09,492056.752411
7,TRAVIS AFB,MULTIPOLYGON (((-121.89653900297888 38.2865679...,94535,CA,3.029397e+08,95232.008421
8,AMERICAN CANYON,MULTIPOLYGON (((-122.20418700285576 38.2096949...,94503,CA,6.931341e+08,136394.695137
9,NOVATO,MULTIPOLYGON (((-122.48655900081091 38.1005269...,94949,CA,4.316054e+08,119395.672078


Rename PO_NAME column with a new name : "Neighborhood"

In [3]:
df = df.rename(columns={'PO_NAME': 'Neighborhood'})

Drop all unecessary columns, in order to get a better view of our table

In [4]:
neighborhoods = df.drop(['the_geom','STATE','Area__','Length__'], axis=1)
neighborhoods

,Neighborhood,ZIP
0,NAPA,94558
1,FAIRFIELD,94533
2,DIXON,95620
3,SONOMA,95476
4,NAPA,94559
5,PETALUMA,94954
6,RIO VISTA,94571
7,TRAVIS AFB,94535
8,AMERICAN CANYON,94503
9,NOVATO,94949


In [5]:
print('The dataframe has {} neighborhoods (not yet compiled).'.format(
        neighborhoods.shape[0])
    )

The dataframe has 187 neighborhoods (not yet compiled).


In [6]:
print('The dataframe has the following number of UNIQUE neighborhoods:')
neighborhoods['Neighborhood'].nunique()

The dataframe has the following number of UNIQUE neighborhoods:


97

In [7]:
print('The dataframe has the following number of UNIQUE ZIP codes:')
neighborhoods['ZIP'].nunique()

The dataframe has the following number of UNIQUE ZIP codes:


187

Thanks to this analysis, we can easily notice that our dataset has 187 rows and 2 columns : Neighborhood & ZIP. In the Neighborhood column we have several repetitions (e.g. NOVATO rows 9 & 10) because we have only 97 unique neighborhoods in the Bay Area. We can also notice that we have 187 ZIP codes. 
Thus, we can conclude by saying that one Neighborhood has several ZIP codes associated. This information should be kept in mind when we will import the different information. We should rather focus on importing data based on the ZIP codes rather than based on the name of the Neighborhood.


Our database is now completely cleaned, we can start playing a little bit with Geopy, running some analysis on our data and all different functionalities. 

#### Use geopy library to get the latitude and longitude values of New York City.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>sf_explorer</em>, as shown below.


In [8]:
address = 'San Fransisco, CA'

geolocator = Nominatim(user_agent="sf_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Francisco are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Francisco are -0.8965048, -78.6549013.


Defining longitude and latitude for each different postal code using geocoder

In [9]:
neighborhoods['Latitude'] = None
neighborhoods['Longitude'] = None
neighborhoods

,Neighborhood,ZIP,Latitude,Longitude
0,NAPA,94558,None,None
1,FAIRFIELD,94533,None,None
2,DIXON,95620,None,None
3,SONOMA,95476,None,None
4,NAPA,94559,None,None
5,PETALUMA,94954,None,None
6,RIO VISTA,94571,None,None
7,TRAVIS AFB,94535,None,None
8,AMERICAN CANYON,94503,None,None
9,NOVATO,94949,None,None


In [10]:
for i, postal_code in enumerate(neighborhoods['ZIP']):
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, CA'.format(postal_code))
        lat_lng_coords = g.latlng
    
    if lat_lng_coords:
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    
    neighborhoods.loc[i, 'Latitude'] = latitude
    neighborhoods.loc[i, 'Longitude'] = longitude

neighborhoods

,Neighborhood,ZIP,Latitude,Longitude
0,NAPA,94558,38.3282,-122.328
1,FAIRFIELD,94533,38.2532,-122.046
2,DIXON,95620,38.4476,-121.826
3,SONOMA,95476,38.3124,-122.484
4,NAPA,94559,38.3013,-122.29
5,PETALUMA,94954,38.2648,-122.641
6,RIO VISTA,94571,38.1593,-121.695
7,TRAVIS AFB,94535,38.28,-121.938
8,AMERICAN CANYON,94503,38.1627,-122.26
9,NOVATO,94949,38.0698,-122.537


Printing Bay-Area Map

In [11]:
map_bay_area = folium.Map(location=[latitude, longitude], zoom_start=8)

for lat, lng, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        parse_html=False).add_to(map_bay_area)
    
map_bay_area

We managed to get a great map of the Bay area and all the key cities and neighborhoods composing the region. We will now use Foursquare in order to move further with our analysis. 

### Foursquare and importing credentials

In [12]:
CLIENT_ID = 'K23DSBS00GSPJIUSF3VQGCCLLLHIOUSJ0244UPB41GFV4DZL' 
CLIENT_SECRET = 'OJ2S52HI3NQGDZL2IM2JZGX11OBYEG31CKBQZXWV2V3TN4RA'
VERSION = '20201207'
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('VERSION:' + VERSION)

Your credentails:
CLIENT_ID: K23DSBS00GSPJIUSF3VQGCCLLLHIOUSJ0244UPB41GFV4DZL
CLIENT_SECRET:OJ2S52HI3NQGDZL2IM2JZGX11OBYEG31CKBQZXWV2V3TN4RA
VERSION:20201207


Let's get all the different venue for each neighborhood by first defining the function that we will use ! 

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

For each of the neighborhood and borough, let's now run the above function to get all venues in the Bay Area


In [15]:
bay_area_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                )

NAPA
FAIRFIELD
DIXON
SONOMA
NAPA
PETALUMA
RIO VISTA
TRAVIS AFB
AMERICAN CANYON
NOVATO
NOVATO
BIRDS LANDING
VALLEJO
BENICIA
VALLEJO
VALLEJO
NOVATO
VALLEJO
NICASIO
OAKLEY
CROCKETT
PORT COSTA
SUISUN CITY
SAN FRANCISCO
PITTSBURG
SAN RAFAEL
CONCORD
RODEO
MARTINEZ
HERCULES
SAN GERONIMO
LAGUNITAS
ALAMEDA
ANTIOCH
SAN ANSELMO
BRENTWOOD
SAN FRANCISCO
CONCORD
FAIRFAX
WOODACRE
FOREST KNOLLS
WALNUT CREEK
PINOLE
RICHMOND
CONCORD
SAN PABLO
SAN RAFAEL
ANTIOCH
EL SOBRANTE
OAKLAND
PLEASANT HILL
CONCORD
GREENBRAE
SAN FRANCISCO
LAFAYETTE
CLAYTON
RICHMOND
RICHMOND
LARKSPUR
SAN QUENTIN
EL CERRITO
CORTE MADERA
WALNUT CREEK
BERKELEY
SAN FRANCISCO
MILL VALLEY
ORINDA
BERKELEY
BERKELEY
BYRON
STINSON BEACH
ALBANY
BERKELEY
SAN FRANCISCO
WALNUT CREEK
BERKELEY
BERKELEY
BERKELEY
ALAMO
BERKELEY
SAUSALITO
MORAGA
BELVEDERE TIBURON
SAN FRANCISCO
BERKELEY
OAKLAND
OAKLAND
OAKLAND
LIVERMORE
EMERYVILLE
DIABLO
DANVILLE
DANVILLE
SAN FRANCISCO
OAKLAND
SAN FRANCISCO
OAKLAND
SAN RAMON
OAKLAND
OAKLAND
CASTRO VALLEY
SAN FRANCISCO
S

In [16]:
print(bay_area_venues.shape)
bay_area_venues.head()

(5121, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,NAPA,38.328225,-122.328115,Alston Park,38.327588,-122.332593,Trail
1,FAIRFIELD,38.253238,-122.046009,Chez Soul,38.249759,-122.046145,Southern / Soul Food Restaurant
2,FAIRFIELD,38.253238,-122.046009,Joe's Buffet,38.249450,-122.044118,Sandwich Place
3,FAIRFIELD,38.253238,-122.046009,China Palace,38.249492,-122.045125,Chinese Restaurant
4,FAIRFIELD,38.253238,-122.046009,Rustwater Kitchen & Taproom,38.249426,-122.043313,Beer Bar


Let's see how many venues appeared for each neighborhood

In [17]:
bay_area_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ALAMEDA,91,91,91,91,91,91
ALAMO,2,2,2,2,2,2
ALBANY,48,48,48,48,48,48
ALVISO,5,5,5,5,5,5
AMERICAN CANYON,3,3,3,3,3,3
ANTIOCH,7,7,7,7,7,7
ATHERTON,5,5,5,5,5,5
BELMONT,50,50,50,50,50,50
BELVEDERE TIBURON,16,16,16,16,16,16


In [18]:
print('There are {} uniques categories.'.format(len(bay_area_venues['Venue Category'].unique())))

There are 374 uniques categories.


We can thus notice that there is a very large number of unique categories within the different venues related to the Bay Area (378)

We can now notice that we have a total of 97 neighborhood in our dataset, and each of these neighborhood have an associated number of venues. Let's now further analyze these venues by determining the kind of venue (Art, Restaurant, Monument, video, clubs, bar...) in each neighborhood : 

In [19]:
# one hot encoding
bay_area_onehot = pd.get_dummies(bay_area_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bay_area_onehot['Neighborhood'] = bay_area_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bay_area_onehot.columns[-1]] + list(bay_area_onehot.columns[:-1])
bay_area_onehot = bay_area_onehot[fixed_columns]

bay_area_onehot.head()

,Yoga Studio,ATM,Acai House,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Alternative Healer,American Restaurant,Amphitheater,Andhra Restaurant,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Baseball Diamond,College Gym,College Library,College Track,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Observatory,Office,Opera House,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Post Office,Print Shop,Pub,Public Art,Public Bathroom,Ra

In [20]:
bay_area_onehot.shape

(5121, 374)

The size of the new dataframe is coherent as we can see that we have 5,178 rows corresponding to the 5,178 venues and 378 columns corresponding to the 378 unique categories. 

In [21]:
bay_area_grouped = bay_area_onehot.groupby('Neighborhood').mean().reset_index()
bay_area_grouped

,Neighborhood,Yoga Studio,ATM,Acai House,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Alternative Healer,American Restaurant,Amphitheater,Andhra Restaurant,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Baseball Diamond,College Gym,College Library,College Track,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Observatory,Office,Opera House,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Post Office,Print Shop,Pub,Public Art,Public Bathroom,Ra

In [22]:
bay_area_grouped.shape

(94, 374)

In this new dataframe, we can notice that we still have the previous 378 columns. But this time we only have 94 rows, this can be explained by the grouping of identical neighborhoods with eachother.
Let's now take a look at the TOP 5 venues for each neighborhood

In [23]:
num_top_venues = 5

for hood in bay_area_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bay_area_grouped[bay_area_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALAMEDA----
                 venue  freq
0          Coffee Shop  0.03
1   Mexican Restaurant  0.03
2  American Restaurant  0.03
3   Italian Restaurant  0.03
4           Nail Salon  0.03


----ALAMO----
               venue  freq
0        Art Gallery   0.5
1       Home Service   0.5
2        Yoga Studio   0.0
3          Nightclub   0.0
4  Other Repair Shop   0.0


----ALBANY----
               venue  freq
0    Thai Restaurant  0.06
1     Breakfast Spot  0.04
2                Bar  0.04
3       Liquor Store  0.04
4  French Restaurant  0.04


----ALVISO----
                venue  freq
0  Mexican Restaurant   0.6
1          Food Truck   0.2
2               Plaza   0.2
3         Yoga Studio   0.0
4           Newsstand   0.0


----AMERICAN CANYON----
          venue  freq
0           ATM  0.33
1  Tennis Court  0.33
2      Pharmacy  0.33
3     Newsstand  0.00
4  Optical Shop  0.00


----ANTIOCH----
                venue  freq
0          Playground  0.29
1                 Gym  0.14
2       

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bay_area_grouped['Neighborhood']

for ind in np.arange(bay_area_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bay_area_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALAMEDA,Nail Salon,Italian Restaurant,Coffee Shop,Mexican Restaurant,American Restaurant,Thai Restaurant,Pharmacy,Dessert Shop,Asian Restaurant,New American Restaurant
1,ALAMO,Art Gallery,Home Service,Women's Store,Farm,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop
2,ALBANY,Thai Restaurant,Bar,Coffee Shop,French Restaurant,Café,Pet Store,Gym / Fitness Center,Indian Restaurant,Breakfast Spot,Liquor Store
3,ALVISO,Mexican Restaurant,Plaza,Food Truck,Women's Store,Farm,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
4,AMERICAN CANYON,ATM,Tennis Court,Pharmacy,Women's Store,Farm,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor


In [26]:
# set number of clusters
k = 50

bay_area_grouped_clustering = bay_area_grouped.drop('Neighborhood', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=1).fit(bay_area_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50] 

array([22, 16, 22, 10, 25, 27,  0, 19, 30, 22, 22,  1, 46, 28, 14, 22,  2,
       22, 29, 21, 19, 22,  9, 38, 19, 22, 31, 41, 22, 24, 26, 19, 22, 22,
        1,  7, 22,  5, 22, 22, 22, 44, 22, 22, 22, 39, 18, 37, 12, 22],
      dtype=int32)

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bay_area_merged = neighborhoods
bay_area_merged = bay_area_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
bay_area_merged.dropna(inplace=True)


In [28]:
print(bay_area_merged.shape)
bay_area_merged['Cluster Labels'] = bay_area_merged['Cluster Labels'].astype(int)
bay_area_merged

(184, 15)


,Neighborhood,ZIP,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,NAPA,94558,38.3282,-122.328,22,Wine Bar,Mexican Restaurant,Italian Restaurant,American Restaurant,Bed & Breakfast,Hotel,Wine Shop,Sushi Restaurant,Steakhouse,Coffee Shop
1,FAIRFIELD,94533,38.2532,-122.046,24,Theater,Southern / Soul Food Restaurant,Sandwich Place,Mexican Restaurant,Mediterranean Restaurant,Chinese Restaurant,Beer Bar,Sushi Restaurant,Falafel Restaurant,Event Service
2,DIXON,95620,38.4476,-121.826,38,Mexican Restaurant,Bar,Sushi Restaurant,Bistro,Bakery,Tea Room,Burger Joint,Women's Store,Eye Doctor,Fabric Shop
3,SONOMA,95476,38.3124,-122.484,48,Mexican Restaurant,Hotel Bar,Mobile Phone Shop,Taco Place,Liquor Store,Bed & Breakfast,French Restaurant,Spa,Snack Place,Thrift / Vintage Store
4,NAPA,94559,38.3013,-122.29,22,Wine Bar,Mexican Restaurant,Italian Restaurant,American Restaurant,Bed & Breakfast,Hotel,Wine Shop,Sushi Restaurant,Steakhouse,Coffee Shop
5,PETALUMA,94954,38.2648,-122.641,34,Construction & Landscaping,Gym,Playground,Coffee Shop,Burger Joint,Cosmetics Shop,Mexican Restaurant,Thai Restaurant,Plaza,Historic Site
6,RIO VISTA,94571,38.1593,-121.695,43,American Restaurant,Sandwich Place,Fast Food Restaurant,Fried Chicken Joint,Gym,Coffee Shop,Grocery Store,Mexican Restaurant,Fishing Store,Auto Workshop
7,TRAVIS AFB,94535,38.28,-121.938,20,Playground,Bowling Alley,Hotel,Movie Theater,Shopping Mall,Park,Falafel Restaurant,Ethiopian Restaurant,Event Service,Event Space
8,AMERICAN CANYON,94503,38.1627,-122.26,25,ATM,Tennis Court,Pharmacy,Women's Store,Farm,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
9,NOVATO,94949,38.0698,-122.537,1,Mexican Restaurant,Pizza Place,Bakery,Bar,Gas Station,Rental Car Location,Gym,Sandwich Place,Italian Restaurant,Breakfast Spot


In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bay_area_merged['Latitude'], bay_area_merged['Longitude'], bay_area_merged['Neighborhood'], bay_area_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Let's now have list of all different clusters : I only put the first 5 clusters, but we could easily focus on the other by changing the figure associated. 

In [37]:
bay_area_merged.loc[bay_area_merged['Cluster Labels'] == 1, bay_area_merged.columns[[1] + list(range(5, bay_area_merged.shape[1]))]]


,ZIP,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,94949,Mexican Restaurant,Pizza Place,Bakery,Bar,Gas Station,Rental Car Location,Gym,Sandwich Place,Italian Restaurant,Breakfast Spot
10,94945,Mexican Restaurant,Pizza Place,Bakery,Bar,Gas Station,Rental Car Location,Gym,Sandwich Place,Italian Restaurant,Breakfast Spot
16,94947,Mexican Restaurant,Pizza Place,Bakery,Bar,Gas Station,Rental Car Location,Gym,Sandwich Place,Italian Restaurant,Breakfast Spot
35,94513,Pizza Place,Mexican Restaurant,American Restaurant,Sandwich Place,Bar,Shipping Store,Gas Station,Bank,Laundromat,Bagel Shop
45,94806,Mexican Restaurant,Supermarket,Pizza Place,Hot Dog Joint,Dessert Shop,Liquor Store,Thai Restaurant,Optical Shop,ATM,Fried Chicken Joint
136,94541,Mexican Restaurant,Pizza Place,Bar,Coffee Shop,Italian Restaurant,Chinese Restaurant,Sushi Restaurant,Park,Cupcake Shop,Ice Cream Shop
138,94542,Mexican Restaurant,Pizza Place,Bar,Coffee Shop,Italian Restaurant,Chinese Restaurant,Sushi Restaurant,Park,Cupcake Shop,Ice Cream Shop
139,94544,Mexican Restaurant,Pizza Place,Bar,Coffee Shop,Italian Restaurant,Chinese Restaurant,Sushi Restaurant,Park,Cupcake Shop,Ice Cream Shop
141,94545,Mexican Restaurant,Pizza Place,Bar,Coffee Shop,Italian Restaurant,Chinese Restaurant,Sushi Restaurant,Park,Cupcake Shop,Ice Cream Shop
143,94080,Mexican Restaurant,Coffee Shop,Japanese Restaurant,Sandwich Place,Italian Restaurant,Diner,Gas Station,Rental Car Location,Thai Restaurant,Chinese Restaurant


In [38]:
bay_area_merged.loc[bay_area_merged['Cluster Labels'] == 22, bay_area_merged.columns[[1] + list(range(5, bay_area_merged.shape[1]))]]


,ZIP,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,94558,Wine Bar,Mexican Restaurant,Italian Restaurant,American Restaurant,Bed & Breakfast,Hotel,Wine Shop,Sushi Restaurant,Steakhouse,Coffee Shop
4,94559,Wine Bar,Mexican Restaurant,Italian Restaurant,American Restaurant,Bed & Breakfast,Hotel,Wine Shop,Sushi Restaurant,Steakhouse,Coffee Shop
12,94591,Park,Bakery,Gas Station,Pet Store,Café,Sporting Goods Shop,Korean Restaurant,Church,Chinese Restaurant,Sushi Restaurant
13,94510,Italian Restaurant,Pizza Place,Burger Joint,Restaurant,Sushi Restaurant,Baseball Field,Bank,Bakery,Tea Room,Salon / Barbershop
14,94592,Park,Bakery,Gas Station,Pet Store,Café,Sporting Goods Shop,Korean Restaurant,Church,Chinese Restaurant,Sushi Restaurant
15,94589,Park,Bakery,Gas Station,Pet Store,Café,Sporting Goods Shop,Korean Restaurant,Church,Chinese Restaurant,Sushi Restaurant
17,94590,Park,Bakery,Gas Station,Pet Store,Café,Sporting Goods Shop,Korean Restaurant,Church,Chinese Restaurant,Sushi Restaurant
23,94103,Coffee Shop,Park,Café,Pizza Place,Italian Restaurant,Bakery,Thai Restaurant,Mexican Restaurant,Cosmetics Shop,Boutique
25,94903,Coffee Shop,Mexican Restaurant,Thai Restaurant,Ice Cream Shop,Sandwich Place,Bar,Indie Movie Theater,Italian Restaurant,Nightclub,American Restaurant
26,94520,Mexican Restaurant,Sandwich Place,Coffee Shop,Italian Restaurant,Café,Pizza Place,Mediterranean Restaurant,Bubble Tea Shop,Playground,Bank


In [39]:
bay_area_merged.loc[bay_area_merged['Cluster Labels'] == 40, bay_area_merged.columns[[1] + list(range(5, bay_area_merged.shape[1]))]]


,ZIP,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,94801,Mexican Restaurant,Pharmacy,Convenience Store,Coffee Shop,Fried Chicken Joint,Furniture / Home Store,Bagel Shop,Sandwich Place,Thrift / Vintage Store,Chinese Restaurant
56,94805,Mexican Restaurant,Pharmacy,Convenience Store,Coffee Shop,Fried Chicken Joint,Furniture / Home Store,Bagel Shop,Sandwich Place,Thrift / Vintage Store,Chinese Restaurant
57,94804,Mexican Restaurant,Pharmacy,Convenience Store,Coffee Shop,Fried Chicken Joint,Furniture / Home Store,Bagel Shop,Sandwich Place,Thrift / Vintage Store,Chinese Restaurant


In [40]:
bay_area_merged.loc[bay_area_merged['Cluster Labels'] == 3, bay_area_merged.columns[[1] + list(range(5, bay_area_merged.shape[1]))]]


,ZIP,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
144,94587,Park,Women's Store,Farm,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory


In [41]:
bay_area_merged.loc[bay_area_merged['Cluster Labels'] == 4, bay_area_merged.columns[[1] + list(range(5, bay_area_merged.shape[1]))]]


,ZIP,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,94973,Baseball Field,Women's Store,Farmers Market,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant
